### Analyse concise du modèle **« Volatility Is (Mostly) Path-Dependent »**

*(Julien Guyon & Jordan Lekeufack, Quantitative Finance 2023)*

| Idée-clé                                              | Résumé                                                                                                                                                               |
| ----------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Constat empirique**                                 | Jusqu’à **90 %** de la variance du VIX s’explique par les seuls retours passés du S\&P 500. La volatilité est donc quasi totalement endogène. ([tandfonline.com][1]) |
| **Formulation continue**                              |                                                                                                                                                                      |
| Spot                                                  | $ \displaystyle \tfrac{dS_t}{S_t}= \sigma_t\,dW_t$  (sous Q)                                                                                                         |
| Vol                                                   | $ \sigma_t = \beta_0 + \beta_1 R^{(1)}_t + \beta_2\sqrt{R^{(2)}_t}$                                                                                                  |
| Mémoire — tendance                                    | $ R^{(1)}_t = \int_0^t \lambda_1 e^{-\lambda_1(t-u)}\,d\!\ln S_u$                                                                                                    |
| Mémoire — « variance »                                | $ R^{(2)}_t = \int_0^t \lambda_2 e^{-\lambda_2(t-u)} (d\!\ln S_u)^2$                                                                                                 |
| Markov 4 facteurs                                     | Deux horizons $(\lambda_{\bullet,0},\lambda_{\bullet,1})$ pour chaque intégrale → modèle « 4-FPDV » multi-échelles, Leverage + Zumbach capturés.                     |
| **Propriétés**                                        | • Modèle **complet** (un seul Brownien) → hedge Δ-only                                                                                                               |
| • Positivité de σ si $\lambda_2<2\lambda_1$.          |                                                                                                                                                                      |
| • Simulation ultra-rapide, pas de vol-de-vol imposé.  |                                                                                                                                                                      |

---

#### Implémentation (pipeline desk-production)

1. **Estimation historique (P-PDV)**

   * Retours $r_t=\ln S_t-\ln S_{t-1}$.
   * Exponentials : $R^{(1)},R^{(2)}$ via EWMA (λ en jours).
   * Régression linéaire (ou Ridge/NN) : $\sigma_{\text{cible}}$ = VIX ou RV → $(\beta_0,\beta_1,\beta_2,\lambda)$.

2. **Calibration risk-neutral (Q-PDV)**

   * Conserver la forme, ajuster $(\beta,\lambda)$ pour **surface SPX + smiles VIX** (least-squares, adjoint-MC).

3. **Monte-Carlo / PDE pricing** (Euler, Δ=1 j typ.)

   ```
   S_{t+Δ}=S_t·exp(σ_t√ΔZ−½σ_t²Δ)
   R¹_{t+Δ}=e^{-λ₁Δ}R¹_t+λ₁Δr_{t+Δ}/Δ
   R²_{t+Δ}=e^{-λ₂Δ}R²_t+λ₂Δr_{t+Δ}²/Δ
   σ_{t+Δ}=β₀+β₁R¹_{t+Δ}+β₂√R²_{t+Δ}
   ```

   * GPU-friendly, aucune Cholesky.
   * Si besoin, ajouter un petit OU « vol-de-vol » (PDSV) pour les chocs exogènes.

4. **Back-test & stress**

   * R² ≈ 0,7 pour la RV 1-5 j sur indices majeurs.&#x20;

---

#### Applications directes — **

| Cas d’usage                                            | Bénéfice PDV                                                                                              |
| ------------------------------------------------------ | --------------------------------------------------------------------------------------------------------- |
| **Exotiques equity** (autocall, cliquet, corridor-var) | Vol dépend de la trajectoire → pricing plus juste des triggers / path risk.                               |
| **Produits VIX & dispersion**                          | Calibration conjointe SPX/VIX : même modèle pour librairie delta-vol.                                     |
| **Hedge & Greeks**                                     | Modèle complet → Δ-hedge unique, vega-slippage réduit vs SV classiques.                                   |
| **Hybridisation taux/FX**                              | Vol endogène limite l’usage de Brownien supplémentaires → couplage plus stable dans les modèles hybrides. |
| **VaR / ES intraday**                                  | Simulations rapides + Zumbach/leverage capturés → scénarios réalistes.                                    |
| **Stratégies vol-target & overlay**                    | Prédiction fiable de la RV courte → ajustement dynamique de la cible de variance.                         |

---

#### Points de vigilance

* **Données** : H-F requises si λ < 1 j.
* **Stabilité** : β₁ (leverage) sensible aux régimes ; recaler régulièrement.
* **Exogène** : événements macro violents → garder composante SV résiduelle.



[1]: https://www.tandfonline.com/doi/abs/10.1080/14697688.2023.2221281?utm_source=chatgpt.com "Volatility is (mostly) path-dependent: Quantitative Finance"


## Construction détaillée du modèle PDV (« Volatility Is (Mostly) Path-Dependent », Guyon-Lekeufack)

### 1.  Intuition empirique

La variance instantanée $\sigma_t^2$ d’un indice (SPX) est expliquée **quasi entièrement** par :

1. une moyenne exponentielle des **retours signés** ($R^{(1)}$) ;
2. une moyenne exponentielle des **retours au carré** ($R^{(2)}$).

Les séries révèlent **deux horizons de mémoire** (intra-semaine vs \~1 an). D’où l’idée de superposer deux exponentielles pour chaque intégrale et d’obtenir **4 facteurs**.&#x20;

---

### 2.  Spécification continue

| Élément                     | Formule (4-FPDV)                                                                                                                                                                             |
| --------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Spot**                    | $\displaystyle\frac{dS_t}{S_t}= \sigma_t\,dW_t$                                                                                                                                              |
| **Volatilité**              | $\displaystyle \sigma_t=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$                                                                                                                        |
| **Kernels**                 | $K_{n}(\tau)=(1-\theta_n)\lambda_{n,0}e^{-\lambda_{n,0}\tau}+\theta_n\lambda_{n,1}e^{-\lambda_{n,1}\tau}$                                                                                    |
| **Facteurs**                | $R_{n,t}= (1-\theta_n)R_{n,0,t}+\theta_n R_{n,1,t}$                                                                                                                                          |
| **Dynamiques Markoviennes** | $\displaystyle\begin{cases} dR_{1,j,t}= \lambda_{1,j}\bigl(\sigma_t\,dW_t-R_{1,j,t}\,dt\bigr)\\ dR_{2,j,t}= \lambda_{2,j}\bigl(\sigma_t^{2}-R_{2,j,t}\bigr)dt \end{cases}\qquad j\in\{0,1\}$ |

**Dimensions :** 5D Markov ($S$ + 4 facteurs). Non explosion et unicité fortes démontrées pour toutes valeurs de paramètres. ([arxiv.org][1])

*Conditions pratiques :* $\beta_0,\beta_2,\lambda_{n,j}\ge0$, $\beta_1\le0$ (effet leverage), et pour garantir $\sigma_t\ge0$ on prend souvent $\lambda_{2,\bullet}<2\lambda_{1,\bullet}$.

---

### 3.  Paramètres et sens économique

| Paramètre         | Impact                                             |
| ----------------- | -------------------------------------------------- |
| $\beta_0$         | niveau de base (vol plancher)                      |
| $\beta_1<0$       | **Leverage** : chute du spot ↑ vol                 |
| $\beta_2>0$       | convexité (cluster de volatilité)                  |
| $\lambda_{1,0/1}$ | vitesses de **mémoire sur retours** (jours ↔ mois) |
| $\lambda_{2,0/1}$ | vitesses de **mémoire sur var. passée**            |
| $\theta_{1,2}$    | pondération long / court terme                     |

---

### 4.  Implémentation numérique (lib pricers desk)

#### 4.1  Pré-calcul des facteurs (back-filling)

En temps discret $t_k=k\Delta$ (Δ = 1 jour ou 5 min) :

```python
R1_j[k] = exp(-λ1_j Δ)*R1_j[k-1] + λ1_j * r[k]
R2_j[k] = exp(-λ2_j Δ)*R2_j[k-1] + λ2_j * r[k]**2
σ[k]    = β0 + β1*R1[k] + β2*sqrt(R2[k])
```

Processus **O(1)**, parallélisable GPU.

#### 4.2  Simulation / pricing

Boucle Euler-lognormale :

$$
S_{k+1}=S_k\exp\!\bigl(\sigma_k\sqrt{\Delta}Z-\tfrac12\sigma_k^2\Delta\bigr)
$$

puis mise-à-jour des quatre $R_{n,j}$ comme ci-dessus. Stable pour Δ ≤ 1 j.

#### 4.3  Calibration Q (« surface jointe » SPX + VIX)

1. **Initial guess :** λ fixés à ceux estimés sous P, β ajustés.
2. **Loss** = somme pondérée des erreurs prix SPX + VIX futures + VIX options.
3. **Optimisation** (Py-BOBYQA / CMA-ES) sur \~50 k scénarios GPU ⇒ < 30 min pour 10 paramètres. ([arxiv.org][2])
4. **Greeks** via adjoint/pathwise, donc ∂/∂β, ∂/∂λ gratuits.

---

### 5.  Extensions utiles 

| Besoin                     | Ajustement                                                                                    |
| -------------------------- | --------------------------------------------------------------------------------------------- |
| **Sauts macro**            | Ajouter un jump Poisson sur $S_t$ ou directement un choc additif sur $\sigma_t$.              |
| **Cross-asset (rates/FX)** | Utiliser même Brownien $W_t$ couplé à Heston-IR ou log-normal FX pour garder dimension basse. |
| **Stress scripts**         | Forcer β₁ → β₁\*(1+ϵ) pour tester scenarios leverage extrême.                                 |

---

### 6.  Pipeline « prod » 

1. **Data feed** haute fréquence → rolling EWMAs pour tenir à jour $R_{n,j}$.
2. **Job nightly** : re-estimation β, recalibration Q (GPU farm).
3. **Cache** surfaces *SPX/VIX exotiques* + *vol-target* prêtes à 7 h UK.
4. **Real-time risk** : delta unique (complet) ; dérivées CVA via bump-&-reprice (simulation << Heston).

---

### 7.  Points de vigilance

* **Stabilité β₁** : très sensible aux régimes, surveiller breakpoints macro.
* **Δ trop grand** → biais (notamment sur VIX 1m). Garder Δ ≤ 0,2 j pour equity vol surfaces.
* **Positivité** : vérifier $\sigma_t>0$ à chaque pas ; sinon clip minimal (ε ≈ 0,1 vol).

---

**À retenir :** le 4-FPDV convertit une dynamique de volatilité fortement empirique (mémoire courte + longue, leverage, clustering) en un système Markovien compact et calculable, idéal pour **pricing rapide**, **calibration jointe** et **hedging simple** .

[1]: https://arxiv.org/pdf/2307.01319 "On the Guyon–Lekeufack Volatility Model"
[2]: https://arxiv.org/html/2406.02319v1 "Pricing and calibration in the 4-factor path-dependent volatility model"


Empiriquement :

1. **Effet court terme (intra-semaine)**
   L’autocorrélation des retours signés et de la variance passée décroît très vite : après ≈ 3-5 jours elle est déjà quasi nulle.
   ⇒ mémoire « rapide » ≈ quelques jours.

2. **Effet long terme (\~ 1 an)**
   Au-delà, on observe une traîne faible mais persistante : la corrélation reste > 0 sur plusieurs centaines de jours.
   ⇒ mémoire « lente » ≈ 250-300 jours.

Pour représenter **les deux échelles simultanément** on prend, pour chaque noyau de mémoire,

$$
K(\tau)= (1-\theta)\,\lambda_0 e^{-\lambda_0 \tau}\;+\;\theta\,\lambda_1 e^{-\lambda_1 \tau},
$$

*superposition de deux exponentielles* :

* $\lambda_0^{-1}$ calibre la mémoire rapide,
* $\lambda_1^{-1}$ la mémoire lente,
* $\theta$ règle leur poids relatif.

On applique ce mix à :

* $R^{(1)}$ (moyenne exponentielle des **retours signés**)
* $R^{(2)}$ (moyenne exponentielle des **retours au carré**)

→ chaque intégrale se scinde en **deux composantes** (court + long terme).
Au total : $R^{(1)}_{\text{court}}, R^{(1)}_{\text{long}}, R^{(2)}_{\text{court}}, R^{(2)}_{\text{long}}$ — soit **4 facteurs** dans la version Markovienne du modèle.


### « Retour signé » vs « retour au carré »

| Terme                          | Définition mathématique                                                                                                                      | Rôle économique                                                                                                                               |
| ------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------- |
| **Retour signé** $r_t$         | $r_t=\ln\!\bigl(S_t/S_{t-\Delta}\bigr)$  (ou $S_t/S_{t-\Delta}-1$) — il garde son **signe** : positif si le spot monte, négatif s’il baisse. | Porte l’**information directionnelle** : un retour négatif (chute du marché) tend à faire monter la vol (effet leverage).                     |
| **Retour au carré** $r_t^{ 2}$ | $(\ln(S_t/S_{t-\Delta}))^{2}$                                                                                                                | Mesure la **magnitude** du mouvement sans tenir compte du signe ; c’est l’élément de base de la variance réalisée (clustering de volatilité). |

Dans le modèle PDV :

* $R^{(1)}_t$ = moyenne exponentielle des **retours signés** → encode l’effet leverage (β₁ < 0).
* $R^{(2)}_t$ = moyenne exponentielle des **retours au carré** → encode le **niveau** de volatilité passée (β₂ > 0).

Ainsi, la combinaison $\sigma_t = β_0 + β_1 R^{(1)}_t + β_2 \sqrt{R^{(2)}_t}$ capte à la fois la **direction récente** du marché et l’**intensité** de ses fluctuations.


### Définitions précises de $R^{(1)}$ et $R^{(2)}$

#### 1. En temps continu (version 4-facteurs)

$$
\boxed{
\begin{aligned}
R^{(1)}_t &\;=\;(1-\theta_1)R^{(1)}_{\text{court},t} + \theta_1 R^{(1)}_{\text{long},t},\\[3pt]
R^{(2)}_t &\;=\;(1-\theta_2)R^{(2)}_{\text{court},t} + \theta_2 R^{(2)}_{\text{long},t},
\end{aligned}}
$$

avec, pour chaque horizon $j\in\{\text{court},\text{long}\}$,

$$
\boxed{
\begin{aligned}
R^{(1)}_{j,t} &= \int_{0}^{t} \lambda_{1,j}\,e^{-\lambda_{1,j}(t-u)}\,d\!\ln S_u, \\[6pt]
R^{(2)}_{j,t} &= \int_{0}^{t} \lambda_{2,j}\,e^{-\lambda_{2,j}(t-u)}\,\bigl(d\!\ln S_u\bigr)^2.
\end{aligned}}
$$

* **$R^{(1)}$** : **moyenne exponentielle des retours signés** (direction du marché).
* **$R^{(2)}$** : **moyenne exponentielle des retours au carré** (intensité des mouvements).

Chaque intégrale utilise **deux constantes de décroissance**
— $\lambda_{\bullet,\text{court}}^{-1}$ ≈ quelques jours,
— $\lambda_{\bullet,\text{long}}^{-1}$ ≈ un an —
pour refléter les deux horizons de mémoire observés.

---

#### 2. En temps discret (pas $\Delta$, typiquement 1 jour ou 5 min)

Pour éviter de ré-intégrer toute l’historique, on met les intégrales sous forme **récurrente (O(1))** :

$$
\boxed{
\begin{aligned}
R^{(1)}_{j,k} &= e^{-\lambda_{1,j}\Delta}\,R^{(1)}_{j,k-1} + \lambda_{1,j}\,r_k,\\[4pt]
R^{(2)}_{j,k} &= e^{-\lambda_{2,j}\Delta}\,R^{(2)}_{j,k-1} + \lambda_{2,j}\,r_k^{2},
\end{aligned}}
$$

où $r_k = \ln(S_k/S_{k-1})$.

On en déduit la volatilité instantanée :

$$
\boxed{\;\sigma_k \;=\; \beta_0 + \beta_1 R^{(1)}_k + \beta_2 \sqrt{R^{(2)}_k}\;}
$$

---

#### 3. Interprétation rapide

| Terme     | Sens                                                                      | Influence sur $\sigma_t$                                    |
| --------- | ------------------------------------------------------------------------- | ----------------------------------------------------------- |
| $R^{(1)}$ | **“Mémoire directionnelle”** : somme pondérée de tous les retours, signés | $\beta_1<0$  ⇒ baisse récente → vol ↑ (leverage)            |
| $R^{(2)}$ | **“Mémoire de variance”** : somme pondérée de tous les retours au carré   | $\beta_2>0$  ⇒ gros mouvements récents → vol ↑ (clustering) |

Ainsi, $R^{(1)}$ capte l’asymétrie (leverage), tandis que $R^{(2)}$ capte le niveau de volatilité ; leur décomposition « court / long terme » donne les **4 facteurs** du modèle Markovien.


Exact !

* **Directionnel court terme ($R^{(1)}_{\text{court}}$)**
  *Mémoire rapide* des retours signés (demi-vie ≈ 2-5 j) → capte le **leverage** intra-semaine.

* **Directionnel long terme ($R^{(1)}_{\text{long}}$)**
  *Mémoire lente* des retours signés (demi-vie ≈ 6-12 mois) → reflète la pente « bear-ish » de long terme.

* **Niveau (variance) court terme ($R^{(2)}_{\text{court}}$)**
  Moyenne exponentielle des $r^2$ à horizon jours → décrit le **clustering** très court de la vol.

* **Niveau (variance) long terme ($R^{(2)}_{\text{long}}$)**
  Moyenne exponentielle des $r^2$ sur \~1 an → fixe la **régime-vol** (bas, moyen, crise).

Ces quatre processus Markoviens suivent :

$$
\begin{aligned}
dR^{(1)}_{j,t}&=\lambda_{1,j}\!\left(\sigma_t\,dW_t-R^{(1)}_{j,t}\,dt\right),\\
dR^{(2)}_{j,t}&=\lambda_{2,j}\!\left(\sigma_t^{2}-R^{(2)}_{j,t}\right)dt,\qquad j\in\{\text{court},\text{long}\},
\end{aligned}
$$

et la volatilité instantanée est simplement

$$
\sigma_t=\beta_0+\beta_1\bigl[(1-\theta_1)R^{(1)}_{\text{court}}+\theta_1R^{(1)}_{\text{long}}\bigr]
          +\beta_2\sqrt{(1-\theta_2)R^{(2)}_{\text{court}}+\theta_2R^{(2)}_{\text{long}} }.
$$

> **En résumé** :
> • 2 horizons × 2 types de mémoire = **4 facteurs**
> • Les deux $R^{(1)}$ gouvernent l’effet *leverage* (signe),
> • Les deux $R^{(2)}$ gouvernent le *level* de volatilité (amplitude).


### Développement : passage **continu → discret** pour les 4 facteurs

---

#### 1.  Facteur directionnel (exemple court-terme)

Continuous form

$$
R^{(1)}_{c}(t)=\int_{0}^{t}\lambda_{1c}\,e^{-\lambda_{1c}(t-u)}\;d\ln S_u .
\tag{1}
$$

---

##### 1.1  SDE exacte

En dérivant (1) :

$$
\frac{d}{dt}R^{(1)}_{c}(t)=\lambda_{1c}\,d\ln S_t-\lambda_{1c} R^{(1)}_{c}(t)\,dt
\quad\Longrightarrow\quad
dR^{(1)}_{c}= \lambda_{1c}\,d\ln S_t-\lambda_{1c}R^{(1)}_{c}\,dt .
\tag{2}
$$

---

##### 1.2  Solution fermée entre deux dates $t_k,\;t_{k+1}=t_k+\Delta$

L’ODE linéaire (2) donne

$$
R^{(1)}_{c}(t_{k+1}) =
e^{-\lambda_{1c}\Delta}\,R^{(1)}_{c}(t_k) + \lambda_{1c}
\int_{t_k}^{t_{k+1}}e^{-\lambda_{1c}(t_{k+1}-u)}d\ln S_u .
\tag{3}
$$

---

##### 1.3  Recursion pratique

Sur le pas $\Delta$ on note le **log-retour**
$r_{k+1}=\ln S_{k+1}-\ln S_k$.

En supposant que $d\ln S_u$ est (quasi) constant sur l’intervalle :

$$
\int_{t_k}^{t_{k+1}}e^{-\lambda_{1c}(t_{k+1}-u)}d\ln S_u
\;\approx\;
\underbrace{\frac{1-e^{-\lambda_{1c}\Delta}}{\Delta}}_{\beta_{1c}}
\,r_{k+1}.
$$

On obtient la mise à jour **exacte au premier ordre**

$$
\boxed{\,R^{(1)}_{c,k+1}
    =\alpha_{1c}\,R^{(1)}_{c,k}+\beta_{1c}\,r_{k+1}\,},
\qquad
\alpha_{1c}=e^{-\lambda_{1c}\Delta},\;
\beta_{1c}=1-\alpha_{1c}.
\tag{4}
$$

> Dans le code on voit souvent la forme simplifiée
> $\;R^{(1)}_{c,k+1}=e^{-\lambda_{1c}\Delta}R^{(1)}_{c,k}+\lambda_{1c}r_{k+1}$ ;
> elle revient à remplacer $\beta$ par $\lambda\Delta$ (erreur $\mathcal O(\Delta^2)$ négligeable pour $\Delta\le1$ j).

---

#### 2.  Facteur de niveau (variance)

Continu :

$$
R^{(2)}_{c}(t)=\int_0^t\lambda_{2c}e^{-\lambda_{2c}(t-u)}\sigma_u^{2}\,du .
\tag{5}
$$

On obtient l’ODE
$dR^{(2)}_{c}= \lambda_{2c}(\sigma_t^{2}-R^{(2)}_{c})\,dt$.

Entre deux dates :

$$
R^{(2)}_{c,k+1}=e^{-\lambda_{2c}\Delta}R^{(2)}_{c,k}
   +(1-e^{-\lambda_{2c}\Delta})\,\sigma_{k}^{2}.
\tag{6}
$$

*Si l’on travaille sur données historiques on remplace*
$\sigma_{k}^{2}\approx r_{k+1}^{2}/\Delta$.

---

#### 3.  Même logique pour les horizons **long terme**

On répète (4)–(6) avec $\lambda_{1\ell}$ et $\lambda_{2\ell}$.

---

### 4.  Algorithme final (4 facteurs)

| Mise à jour (pas $\Delta$)                                                                                     |
| -------------------------------------------------------------------------------------------------------------- |
| $R^{(1)}_{\text{court},k+1}= \alpha_{1c} R^{(1)}_{\text{court},k} + \beta_{1c} r_{k+1}$                        |
| $R^{(1)}_{\text{long },k+1}= \alpha_{1\ell}R^{(1)}_{\text{long },k}+ \beta_{1\ell}r_{k+1}$                     |
| $R^{(2)}_{\text{court},k+1}= \alpha_{2c} R^{(2)}_{\text{court},k}+ (1-\alpha_{2c})\,\hat{\sigma}_{k}^{2}$      |
| $R^{(2)}_{\text{long },k+1}= \alpha_{2\ell}R^{(2)}_{\text{long },k}+ (1-\alpha_{2\ell})\,\hat{\sigma}_{k}^{2}$ |

avec
$\alpha_{\bullet}=e^{-\lambda_{\bullet}\Delta},\;
\beta_{1\bullet}=1-\alpha_{1\bullet},\;
\hat{\sigma}_{k}^{2}=r_{k+1}^{2}/\Delta$ (hist) ou $\sigma_{k}^{2}$ (simu).

La volatilité instantanée utilisée dans le pricing est alors

$$
\sigma_k=\beta_0+\beta_1\bigl[(1-\theta_1)R^{(1)}_{\text{court},k}+\theta_1R^{(1)}_{\text{long},k}\bigr]
      +\beta_2\sqrt{(1-\theta_2)R^{(2)}_{\text{court},k}+\theta_2R^{(2)}_{\text{long},k}}.
$$

---

**En bref :** chacune des quatre exponentielles est **exactement** recalculée en $O(1)$ grâce aux coefficients $\alpha=e^{-\lambda\Delta}$ et $\beta=1-\alpha$. Cette discrétisation est cohérente avec la solution fermée des ODEs, donc stable et sans biais à l’ordre 1 pour $\Delta\le1$ jour.


### Que sont les $\lambda$ ?

*Coefficients de décroissance* des mémoires exponentielles :

| Symbole            | Mémoire                            | Horizon caractéristique                     | Interprétation économique                                                  |
| ------------------ | ---------------------------------- | ------------------------------------------- | -------------------------------------------------------------------------- |
| $\lambda_{1c}$     | retours **signés** – court terme   | $T_{½}\!\approx\!\ln2/\lambda_{1c}$ = 2-5 j | Rapidité d’apparition / disparition de l’effet **leverage** intraday-hebdo |
| $ \lambda_{1\ell}$ | retours **signés** – long terme    | 6-12 mois                                   | Persistance d’un biais baissier de fond                                    |
| $\lambda_{2c}$     | retours **au carré** – court terme | 1-10 j                                      | Durée des bouffées de vol (vol-spikes)                                     |
| $ \lambda_{2\ell}$ | retours **au carré** – long terme  | 6-18 mois                                   | Vitesse de retour au régime-vol « normal » après crise                     |

*(Le demi-vie $T_{½}$ est souvent plus parlant que $1/\lambda$.*)

---

### Comment les estimer ?  — **Sous P (historique)**

1. **Recherche brutale ou ML léger**

   * Balayer une grille de paires $(\lambda_{1c},\lambda_{1\ell})$ et $(\lambda_{2c},\lambda_{2\ell})$.
   * Pour chaque quadriplet : calculez $R^{(1)}$ & $R^{(2)}$, réglez $(\beta_0,\beta_1,\beta_2)$ par régression sur la RV future (ou VIX).
   * Garder le jeu qui maximise $R^2$ hors échantillon.

2. **Méthode « auto-corrélation cible »**

   * Mesurer la fonction d’auto-corrélation empirique de $|r_t|$ et de $r_t^2$.
   * Ajuster deux exponentielles de la forme $a e^{-\lambda_c \tau} + (1-a) e^{-\lambda_\ell \tau}$ en moindres carrés → donne directement les quatre $\lambda$.

3. **Kalman / EM** (si l’on estime les facteurs latent et les paramètres simultanément).

---

### Calibration **sous Q** (optionnelle)

* Commencez avec les $\lambda$ historiques.
* Recalibrez-les en bloc (ou seulement $\lambda_{2c}$ & $\lambda_{2\ell}$) sur la **surface SPX + VIX** ; ils pilotent la pente temporelle de la vol implicite.
* Le plus souvent on touche peu aux $\lambda_1$ (leverage) et on ajuste surtout $\beta$.

---

### Lecture rapide

* **Petit $\lambda$ ⇒ longue mémoire** : le facteur bouge lentement, stabilise la vol de long terme.
* **Grand $\lambda$ ⇒ mémoire courte** : facteur très réactif, nécessaire pour reproduire les pics intraday/journaliers.
* **Ratio $\lambda_{2}/\lambda_{1}$** (> 1 typiquement) assure la positivité de $\sigma_t$.
* Sensibilité : les smiles lointains dépendent surtout de $\lambda_{2\ell}$; les options courtes d’échéance dépendent de $\lambda_{2c}$.

En pratique, une fois les $\lambda$ fixés, le modèle devient **linéaire** en $(\beta_0,\beta_1,\beta_2)$ et se recalcule en quelques millisecondes — d’où la robustesse de la calibration quotidienne


### Comment les $\lambda$ **codifient** la mémoire court / long terme

*(et pourquoi il en faut exactement quatre)*

| Symbole  | Horizon « 1/ λ » | Demi-vie $T_{½}= \ln2/λ$ | Effet concret dans le modèle                                                                           |
| -------- | ---------------- | ------------------------ | ------------------------------------------------------------------------------------------------------ |
| $λ_{1c}$ | 1-10 jours       | 2-5 j                    | **Leverage flash** : la vol bondit quand le spot chute brusquement, puis s’oublie en quelques séances. |
| $λ_{1ℓ}$ | 6-12 mois        | 100-250 j                | **Biais directionnel lent** : tendance baissière durable ↗ vol implicite de long terme.                |
| $λ_{2c}$ | 1-15 jours       | 3-7 j                    | **Clustering court** : un spike de vol (CPI surprise…) se résorbe en \~ une semaine.                   |
| $λ_{2ℓ}$ | 6-18 mois        | 150-300 j                | **Régime de volatilité** : sortie de crise 2008/2020, retour au niveau de croisière.                   |

---

#### 1.  Du noyau exponentiel à la **mémoire effective**

Une moyenne exponentielle

$$
K_{λ}(\tau)=λe^{-λ\tau}\;\;(τ\ge0)
$$

attribue au retour vieux de $τ$ jours le poids
$w(τ)=e^{-λτ}$.

* **Grand $λ$** ⇒ pente raide ⇒ ne « voit » que le passé proche.
* **Petit $λ$** ⇒ pente molle ⇒ garde trace d’événements anciens.

---

#### 2.  Superposition : 2 horizons × 2 types

$$
\text{Poids total}(τ)=
(1-\theta)\,λ_{c}e^{-λ_{c}τ}
+\theta\,λ_{ℓ}e^{-λ_{ℓ}τ}.
$$

* **$\theta$** règle « le curseur » entre court (0) et long terme (1).
* La **courbe** de poids épouse très bien l’autocorrélation empirique, creusée à l’origine et traînante plus loin.

---

#### 3.  Implémentation **discrète** : 1 ligne de code par facteur

```python
α = exp(-λ * Δ)      # mémoire résiduelle
β = 1 - α            # poids du retour courant
R_next = α * R + β * r             # pour les retours signés
V_next = α * V + (1-α) * sigma**2  # pour les retours²
```

Choisissez $λ$ :

| Cible empirique     | Recette rapide                     |
| ------------------- | ---------------------------------- |
| Spike intraday      | $λ_{2c}= \ln2 / 3\text{ j}^{-1}$   |
| Bridge 3-6 m        | $λ_{2ℓ}= \ln2 / 180\text{ j}^{-1}$ |
| Leverage day-to-day | $λ_{1c}= \ln2 / 2\text{ j}^{-1}$   |
| Leverage macro      | $λ_{1ℓ}= \ln2 / 120\text{ j}^{-1}$ |

---

#### 4.  Impact direct sur les **smiles** et la **courbe de vol**

| Paramètre  | Sourires courts           | Sourires longs                | Term-structure VIX               |
| ---------- | ------------------------- | ----------------------------- | -------------------------------- |
| $λ_{2c}$ ↑ | smiles 1-30 d plus pentus | quasi nul                     | élève VIX front-month            |
| $λ_{2ℓ}$ ↓ | peu d’effet               | creuse / twist smiles 1-3 ans | relève VIX 1-2 ans               |
| $λ_{1c}$ ↑ | accentue skew très court  | nul                           | baisse/relève VIX selon signe β₁ |
| $λ_{1ℓ}$ ↓ | lisse skew court          | accentue skew long            | hausse vol longue                |

---

#### 5.  Pourquoi **quatre** est suffisant

1. Mémoire empirique = **double-exponentielle** → 2 paramètres par type.
2. Deux types (direction vs niveau) → 4 facteurs.
3. Ajout d’un troisième horizon → gain minime, coût CPU + calibration ↑.

---

**À retenir** :

* Les $λ$ ne sont pas « des » vitesses : ce sont *les* boutons qui transforment les retours bruts en une **mémoire pondérée** parfaitement réglable.
* En jouant sur leurs valeurs (et sur $\theta$), on **calibre** la réactivité ultra-courte et la persistance longue durée **sans ajouter de Vol-de-Vol**.
* Un desk n’a qu’à figer les $λ$ (court / long) une fois, puis recalibrer chaque jour les $\beta$ — cycle robuste et rapide.


### Formule « forward-variance » → pont direct entre **SPX** et **VIX**

Pour un horizon $T$ (30 j pour le VIX spot), le futur VIX est

$$
\boxed{F^{\text{VIX}}_0(T)=100
\sqrt{\frac{1}{T}\;
\underbrace{\mathbb E_Q\!\Bigl[\int_{0}^{T}\!\sigma_t^{2}\,dt\Bigr]}_{:=\,\Xi(T)}}}\;.
$$

Dans le 4-FPDV, les deux facteurs de niveau $R^{(2)}_{c},R^{(2)}_{\ell}$ sont des processus OU :

$$
dR^{(2)}_{j,t}= \lambda_{2j}\bigl(\sigma_t^{2}-R^{(2)}_{j,t}\bigr)dt,\qquad j\in\{c,\ell\}.
$$

En prenant l’espérance conditionnelle (Markov, mémoire exponentielle) on obtient exactement :

$$
\Xi(T)=A_0(T)+A_c(T)\,R^{(2)}_{c}(0)+A_\ell(T)\,R^{(2)}_{\ell}(0),
$$

avec
$A_j(T)=\dfrac{1-e^{-\lambda_{2j}T}}{\lambda_{2j}}$(pondérés par $\theta_2$).
**⇒ VIX futur** (et donc les options VIX) se lit **linéairement** dans l’état courant des deux $R^{(2)}$.

---

## Routine de calibration conjointe **SPX + VIX**

> Objectif : trouver $(\beta_0,\beta_1,\beta_2)$ (et éventuellement $λ_{2c},λ_{2\ell}$) qui minimisent
> l’erreur prix sur *SPX vanilles* **et** *futurs/op­tions VIX*.

| Étape                 | Détail pratique                                                                                                                                                                                                                                  |
| --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **1. Fixer les λ**    | Prenez ceux estimés sous P (ou ré-optimisez $λ_{2\bullet}$ uniquement ; toucher à $λ_{1\bullet}$ casse le skew).                                                                                                                                 |
| **2. État courant**   | Calculez $R^{(2)}_{c}(0),R^{(2)}_{\ell}(0)$ à partir des retours des 12 derniers mois (recursion O(1)).                                                                                                                                          |
| **3. Seed linéaire**  | La formule analytique de $\Xi(T)$ rend le problème **linéaire en β** → une régression GLS sur les prix **VIX futures** fournit un premier $(β_0,β_2)$. Fixez provisoirement $β_1$ avec le skew SPX 1 M.                                          |
| **4. Objectif joint** | $\text{Loss}=w_{\text{SPX}}\sum(\text{IV}^{\text{mod}}-\text{IV}^{\text{mkt}})^2 + w_{\text{VIX}}\sum(\Xi^{\text{mod}}-\Xi^{\text{mkt}})^2.$ <br>• SPX : prix via MC (≈ 10 k chemins, GPU) <br>• VIX futures : formule fermée ci-dessus (0 cpu). |
| **5. Optimisation**   | CMA-ES global puis L-BFGS local (10 paramères → < 30 s sur GPU). Tous les gradients sont automatiques ; $\partial \Xi/\partial β$ est analytique.                                                                                                |
| **6. VIX options**    | Prix avec un petit MC **antithétique** où l’on recycle la valeur fermée de $\Xi(T)$ comme control-variate ⇒ erreur<0.3 vol-pt. (On les **laisse dans la fonction de coût** pour verrouiller curvature).                                          |
| **7. Validation**     | < 1 vol-pt RMS sur SPX δ 20-80 % & VIX 1-24 m ; R² > 70 % sur RV\_5d hors-échantillon.                                                                                                                                                           |

---

### Pourquoi la formule ferme accélère vraiment

* Sans elle : il faudrait simuler $\int\sigma_t^2dt$ dans la boucle calib → ×100 plus lent.
* Avec elle : **VIX futures = simple racine carrée d’un polynôme affine** des deux $R^{(2)}$ courants ; les dérivées $\partial/\partial β,λ$ tombent en une ligne.
* Résultat : joint-fit complet SPX+VIX en routine nocturne < 2 min, greeks disponibles en temps réel pour XVA/risk.

---

#### Formule flash (prête à coder)

```python
def vix_future(R2c0, R2l0, beta0, beta2, lam2c, lam2l, theta2, T):
    Ac = (1-np.exp(-lam2c*T))/lam2c
    Al = (1-np.exp(-lam2l*T))/lam2l
    Xi = beta0**2*T + beta2**2*((1-theta2)*Ac*R2c0 + theta2*Al*R2l0)
    return 100*np.sqrt(Xi/T)
```

> **À retenir** :
>
> * La *forward variance* se résume à trois coefficients $(A_0,A_c,A_\ell)$ déterminés par les seuls $λ_{2\bullet}$.
> * Cette linéarité rend la calibration **rapide, stable et parfaitement cohérente** entre SPX et VIX.


**σ\_t dans le modèle PDV = volatilité *instantanée* (spot)**

Voici la version corrigée du petit tableau — sans « trou » — qui situe clairement **σₜ** par rapport aux autres notions :

| Quantité                            | Formule / dynamique                                                               | Nature de la mesure                                                                                            | Utilisation principale                                           |
| ----------------------------------- | --------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------- |
| **σₜ**                              | $\displaystyle \sigma_t = \beta_0 + \beta_1 R^{(1)}_t + \beta_2 \sqrt{R^{(2)}_t}$ | Volatilité **instantanée (spot)**, définie dans le modèle (sous P pour l’estimation, sous Q après calibration) | Simuler $S_t$ via $dS_t/S_t = σ_t\,dW_t$ → base pour RV, IV, VIX |
| **Volatilité réalisée** sur $[0,T]$ | $\sqrt{\tfrac1T \int_0^T σ_u^{2}\,du}$                                            | Mesure **ex post**                                                                                             | Back-tests, contrôle de qualité du modèle                        |
| **Forward variance** $ \Xi(T)$      | $\mathbb E_Q\!\bigl[ \int_0^T σ_u^{2} du \bigr]$                                  | Variance future **sous Q** (donc implicite)                                                                    | Entre directement dans la quote des **futures / options VIX**    |
| **Volatilité implicite (IV)**       | Valeur de σ telle que Black-Scholes reproduit le prix option                      | Objet de marché, dérivée de la loi future de $S_T$                                                             | Surface SPX, calibration quotidienne                             |

> **À retenir** : la formule de **σₜ** décrit la vol instantanée ; toutes les autres volatilités (réalisée, forward / VIX, implicite) sont des agrégats ou des fonctions de ce σₜ.

* **Sous P** (historique) quand on estime le modèle sur les retours passés ;
* **Sous Q** (risk-neutral) après calibration SPX + VIX ; | Sert à simuler $S_t$ via $dS_t/S_t=\sigma_t\,dW_t$ et donc à **produire** :<br>• la volatilité réalisée future ;<br>• les surfaces d’IV (via espérance sous Q). |

---

### Comment relier σ\_t aux autres notions

| Notion                                                | Formule à partir de σ\_t                                                                       | Commentaire                                                                        |
| ----------------------------------------------------- | ---------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------- |
| **Volatilité réalisée** sur \[0,T]                    | $\displaystyle \sqrt{\frac1T\int_0^T \sigma_u^{2}\,du}$                                        | Connue **a-posteriori** ; utilisée pour back‐tests.                                |
| **Forward variance** (clé du VIX)                     | $\displaystyle \Xi(T)=\mathbb E_Q\!\Bigl[\int_0^{T}\sigma_u^{2}\,du\Bigr]$                     | Se calcule analytiquement dans PDV (linéaire en $R^{(2)}$).                        |
| **Volatilité impliquée (IV)** d’une option maturité T | Trouvée numériquement : prix option = $E_Q[\text{payoff}(S_T)]$ où $S_t$ suit $dS_t$ ci-dessus | L’IV est une **fonction** de la loi future de $σ_t$ ; ce n’est pas $σ_t$ lui-même. |

---

**En résumé :**

* La formule que je t’ai donnée décrit l’**état instantané** de la volatilité dans le modèle.
* Une fois σ\_t simulé (ou son espérance intégrée) on en déduit :

  * **réalisée** : intégrale de σ\_t² sur la période,
  * **impliquée / VIX** : espérance sous Q de cette même intégrale.
* Donc σ\_t n’est ni l’IV ni la RV ; c’est la *matière première* dont les deux découlent.
